In [1]:
import pickle
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import re

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swnu_network_functions import (
    swnu_network_hyperparameter_search,
    obtain_SWNUNetwork_input,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [7]:
x_data = obtain_SWNUNetwork_input(
    method="gaussian_random_projection",
    dimension=30,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    k=5,
    features="time_encoding",
    standardise_method=None,
    include_features_in_path=False,
)

x_data["x_data"]["path"].shape

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


torch.Size([5568, 5, 30])

## SWNU Network

In [8]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [9]:
num_epochs = 100
dimensions = [15]
swnu_hidden_dim_sizes_and_sig_depths = [([12], 3), ([10], 4)]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swnu_hidden_dim_sizes_and_sig_depths": swnu_hidden_dim_sizes_and_sig_depths,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "BiLSTM": True,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [12]:
size = 5

## GRP

In [13]:
(
    swnu_network_grp_kfold_5,
    best_swnu_network_grp_kfold_5,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_5_kfold_best_model.csv


In [14]:
swnu_network_grp_kfold_5.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_1331235/543207788.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_5.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.693135   
                                                       0.0005         0.695806   
                                                       0.0010         0.685927   
                                          0.2          0.0001         0.693880   
                                                       0.0005         0.694315   
                                                       0.0010         0.686797   
                           (512, 512)     0.1          0.0001         0.693632   
                                                       0.0005         0.693072   
                                                       0.0010         0.691271   
                                          0.2          0.0001         0.696055   
                                                       0.0005         0.693321   
                                                       0.0010         0.692948   
           (12,)           (256, 256)     0.1          0.0001         0.692637   
                                                       0.0005         0.691333   
                                                       0.0010         0.694439   
                                          0.2          0.0001         0.694004   
                                                       0.0005         0.688723   
                                                       0.0010         0.688661   
                           (512, 512)     0.1          0.0001         0.691892   
                                                       0.0005         0.693010   
                                                       0.0010         0.692762   
                                          0.2          0.0001         0.692451   
                                                       0.0005         0.686114   
                                                       0.0010         0.687294   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.671246   
                                                       0.0005         0.673074   
                                                       0.0010         0.665091   
                                          0.2          0.0001         0.673481   
                                                       0.0005         0.672388   
                                                       0.0010         0.668937   
                           (512, 512)     0.1          0.0001         0.671331   
                                                       0.0005         0.671114   
                                                       0.0010         0.665801   
                                          0.2          0.0001         0.674046   
                                                       0.0005         0.671865   
                                                       0.0010         0.670163   
           (12,)           (256, 256)     0.1          0.0001         0.671742   
                                                       0.0005         0.670782   
                                                       0.0010         0.670509   
                                          0.2          0.0001         0.672661   
                                                       0.0005         0.668669   
                                                       0.0010         0.666276   
                           (512, 512)     0.1          0.0001         0.669711   
                                                       0.0005         0.670823   
                                                       0.0010         0.669697   
                                          0.2      

In [15]:
best_swnu_network_grp_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.692451,0.670081,"[0.7559893522626442, 0.5841733870967741]",0.667488,"[0.7806963958460599, 0.5542802486848398]",0.675136,"[0.7327981651376146, 0.6174746936600959]",None,0.735139,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.690960,0.669098,"[0.7541518386714117, 0.5840441545408931]",0.666470,"[0.7810196560196561, 0.5519203413940256]",0.674605,"[0.7290711009174312, 0.6201385189131593]",None,0.739992,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.693942,0.675254,"[0.7531569452796152, 0.5973516429622364]",0.672552,"[0.7917193426042983, 0.5533848250795094]",0.683542,"[0.7181766055045872, 0.648907831646244]",None,0.736083,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


in path: 0.672431425433941 (2)

in input: 0.6778592725453016 (1)

both: 0.6714778868022625 (3)

In [16]:
best_swnu_network_grp_kfold_5["f1"].mean()

0.6714778868022625

In [17]:
best_swnu_network_grp_kfold_5["precision"].mean()

0.6688368016047314

In [18]:
best_swnu_network_grp_kfold_5["recall"].mean()

0.6777611526298554

In [19]:
np.stack(best_swnu_network_grp_kfold_5["f1_scores"]).mean(axis=0)

array([0.75443271, 0.58852306])

In [20]:
np.stack(best_swnu_network_grp_kfold_5["precision_scores"]).mean(axis=0)

array([0.78447846, 0.55319514])

In [21]:
np.stack(best_swnu_network_grp_kfold_5["recall_scores"]).mean(axis=0)

array([0.72668196, 0.62884035])

# w=11

In [12]:
size = 11

## GRP

In [13]:
(
    swnu_network_grp_kfold_11,
    best_swnu_network_grp_kfold_11,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_11_kfold_best_model.csv


In [14]:
swnu_network_grp_kfold_11.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3220708/3519552416.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_11.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.694812   
                                                       0.0005         0.691768   
                                                       0.0010         0.679031   
                                          0.2          0.0001         0.695061   
                                                       0.0005         0.694750   
                                                       0.0010         0.679155   
                           (512, 512)     0.1          0.0001         0.691954   
                                                       0.0005         0.688661   
                                                       0.0010         0.680833   
                                          0.2          0.0001         0.695930   
                                                       0.0005         0.687418   
                                                       0.0010         0.672321   
           (12,)           (256, 256)     0.1          0.0001         0.698478   
                                                       0.0005         0.688723   
                                                       0.0010         0.678099   
                                          0.2          0.0001         0.695620   
                                                       0.0005         0.688475   
                                                       0.0010         0.678907   
                           (512, 512)     0.1          0.0001         0.699472   
                                                       0.0005         0.692078   
                                                       0.0010         0.677291   
                                          0.2          0.0001         0.695993   
                                                       0.0005         0.685492   
                                                       0.0010         0.683318   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.675692   
                                                       0.0005         0.670981   
                                                       0.0010         0.662616   
                                          0.2          0.0001         0.674634   
                                                       0.0005         0.673231   
                                                       0.0010         0.662309   
                           (512, 512)     0.1          0.0001         0.670878   
                                                       0.0005         0.669304   
                                                       0.0010         0.663601   
                                          0.2          0.0001         0.675602   
                                                       0.0005         0.665095   
                                                       0.0010         0.657395   
           (12,)           (256, 256)     0.1          0.0001         0.677104   
                                                       0.0005         0.669341   
                                                       0.0010         0.660705   
                                          0.2          0.0001         0.675354   
                                                       0.0005         0.669411   
                                                       0.0010         0.660651   
                           (512, 512)     0.1          0.0001         0.678183   
                                                       0.0005         0.670070   
                                                       0.0010         0.660703   
                                          0.2      

In [15]:
best_swnu_network_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.676980,0.661253,"[0.7342432142309461, 0.588263245426467]",0.660096,"[0.7893175074183977, 0.5308747855917667]",0.672958,"[0.6863532110091743, 0.6595631326584976]",None,0.766411,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.679031,0.659173,"[0.7414414414414415, 0.576904176904177]",0.656856,"[0.7783732660781841, 0.5353397172822617]",0.666661,"[0.707855504587156, 0.6254661694192861]",None,0.760345,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.693942,0.677224,"[0.7506832675372002, 0.6037644787644788]",0.674882,"[0.7979341510652034, 0.5518306131451257]",0.687602,"[0.7087155963302753, 0.6664890783164624]",None,0.743766,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


in path: 0.6678837381461014

in input: 0.6446273342007033

both: 0.6658833040507851

In [16]:
best_swnu_network_grp_kfold_11["f1"].mean()

0.6658833040507851

In [17]:
best_swnu_network_grp_kfold_11["precision"].mean()

0.6639450067634899

In [18]:
best_swnu_network_grp_kfold_11["recall"].mean()

0.6757404487201418

In [19]:
np.stack(best_swnu_network_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.74212264, 0.58964397])

In [20]:
np.stack(best_swnu_network_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.78854164, 0.53934837])

In [21]:
np.stack(best_swnu_network_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.70097477, 0.65050613])

# w=20

In [22]:
size = 20

## GRP

In [23]:
(
    swnu_network_grp_kfold_20,
    best_swnu_network_grp_kfold_20,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_20_kfold_best_model.csv


In [24]:
swnu_network_grp_kfold_20.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3220708/1556030676.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_20.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.689842   
                                                       0.0005         0.677229   
                                                       0.0010         0.673874   
                                          0.2          0.0001         0.689096   
                                                       0.0005         0.679652   
                                                       0.0010         0.675986   
                           (512, 512)     0.1          0.0001         0.688475   
                                                       0.0005         0.676483   
                                                       0.0010         0.683566   
                                          0.2          0.0001         0.691084   
                                                       0.0005         0.680957   
                                                       0.0010         0.677850   
           (12,)           (256, 256)     0.1          0.0001         0.691084   
                                                       0.0005         0.671140   
                                                       0.0010         0.670581   
                                          0.2          0.0001         0.689158   
                                                       0.0005         0.675241   
                                                       0.0010         0.668282   
                           (512, 512)     0.1          0.0001         0.697235   
                                                       0.0005         0.689282   
                                                       0.0010         0.666480   
                                          0.2          0.0001         0.693197   
                                                       0.0005         0.681827   
                                                       0.0010         0.667350   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.670453   
                                                       0.0005         0.661547   
                                                       0.0010         0.660467   
                                          0.2          0.0001         0.668098   
                                                       0.0005         0.662610   
                                                       0.0010         0.661562   
                           (512, 512)     0.1          0.0001         0.670506   
                                                       0.0005         0.661751   
                                                       0.0010         0.666755   
                                          0.2          0.0001         0.671680   
                                                       0.0005         0.662909   
                                                       0.0010         0.662594   
           (12,)           (256, 256)     0.1          0.0001         0.672110   
                                                       0.0005         0.658321   
                                                       0.0010         0.657659   
                                          0.2          0.0001         0.671979   
                                                       0.0005         0.662856   
                                                       0.0010         0.654769   
                           (512, 512)     0.1          0.0001         0.677310   
                                                       0.0005         0.672422   
                                                       0.0010         0.652283   
                                          0.2      

In [25]:
best_swnu_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.657223,0.644576,"[0.711619883957974, 0.5775327360441076]",0.646533,"[0.7853928695050191, 0.5076736672051696]",0.660101,"[0.6505160550458715, 0.6696856686201386]",None,0.840005,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.656850,0.646172,"[0.7076385580435127, 0.5847056169636815]",0.650104,"[0.7931648273406906, 0.5070422535211268]",0.664612,"[0.6387614678899083, 0.690463505594033]",None,0.799569,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.690774,0.673559,"[0.7485220554797635, 0.5985966610210501]",0.671231,"[0.7941460276616276, 0.5483156028368794]",0.683443,"[0.707855504587156, 0.6590303676078849]",None,0.816148,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [26]:
best_swnu_network_grp_kfold_20["f1"].mean()

0.6547692519183482

In [27]:
best_swnu_network_grp_kfold_20["precision"].mean()

0.6559558746784188

In [28]:
best_swnu_network_grp_kfold_20["recall"].mean()

0.6693854282241655

In [29]:
np.stack(best_swnu_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.7225935, 0.586945 ])

In [30]:
np.stack(best_swnu_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.79090124, 0.52101051])

In [31]:
np.stack(best_swnu_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.66571101, 0.67305985])

# w=35

In [32]:
size = 35

## GRP

In [33]:
(
    swnu_network_grp_kfold_35,
    best_swnu_network_grp_kfold_35,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swnu_network_grp_focal_2_35_kfold_best_model.csv


In [34]:
swnu_network_grp_kfold_35.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

/tmp/ipykernel_3220708/2654077240.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swnu_network_grp_kfold_35.groupby(


accuracy  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.695247   
                                                       0.0005         0.689842   
                                                       0.0010         0.677415   
                                          0.2          0.0001         0.692513   
                                                       0.0005         0.690774   
                                                       0.0010         0.684560   
                           (512, 512)     0.1          0.0001         0.693880   
                                                       0.0005         0.691954   
                                                       0.0010         0.687108   
                                          0.2          0.0001         0.688040   
                                                       0.0005         0.684809   
                                                       0.0010         0.678223   
           (12,)           (256, 256)     0.1          0.0001         0.691892   
                                                       0.0005         0.683131   
                                                       0.0010         0.682075   
                                          0.2          0.0001         0.692948   
                                                       0.0005         0.687729   
                                                       0.0010         0.672693   
                           (512, 512)     0.1          0.0001         0.696614   
                                                       0.0005         0.685492   
                                                       0.0010         0.682759   
                                          0.2          0.0001         0.693942   
                                                       0.0005         0.685803   
                                                       0.0010         0.688350   

                                                                            f1  \
dimensions swnu_hidden_dim ffn_hidden_dim dropout_rate learning_rate             
15         (10,)           (256, 256)     0.1          0.0001         0.669219   
                                                       0.0005         0.667099   
                                                       0.0010         0.658967   
                                          0.2          0.0001         0.668177   
                                                       0.0005         0.670093   
                                                       0.0010         0.664378   
                           (512, 512)     0.1          0.0001         0.670766   
                                                       0.0005         0.668438   
                                                       0.0010         0.667354   
                                          0.2          0.0001         0.666828   
                                                       0.0005         0.664723   
                                                       0.0010         0.660984   
           (12,)           (256, 256)     0.1          0.0001         0.667476   
                                                       0.0005         0.664349   
                                                       0.0010         0.665919   
                                          0.2          0.0001         0.669462   
                                                       0.0005         0.668402   
                                                       0.0010         0.655690   
                           (512, 512)     0.1          0.0001         0.673208   
                                                       0.0005         0.665343   
                                                       0.0010         0.664855   
                                          0.2      

In [35]:
best_swnu_network_grp_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,seed,BiLSTM,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.670084,0.648238,"[0.7358997314234557, 0.5605759682224428]",0.646070,"[0.7672682016179216, 0.5248721524872152]",0.654244,"[0.7069954128440367, 0.6014917421417155]",None,0.850115,...,1,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.670829,0.655063,"[0.7288083538083538, 0.5813181602655287]",0.654215,"[0.7847222222222222, 0.5237078171721486]",0.666751,"[0.6803325688073395, 0.6531699520511455]",None,0.854967,...,12,True,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.677167,0.663768,"[0.7308887507768801, 0.5966464834653005]",0.663912,"[0.7978290366350068, 0.5299958626396359]",0.678392,"[0.6743119266055045, 0.6824720298348428]",None,0.830840,...,123,True,focal,2,True,5,Conv1d,None,concatenation,64


In [36]:
best_swnu_network_grp_kfold_35["f1"].mean()

0.655689574660327

In [37]:
best_swnu_network_grp_kfold_35["precision"].mean()

0.6547325487956918

In [38]:
best_swnu_network_grp_kfold_35["recall"].mean()

0.6664622720474308

In [39]:
np.stack(best_swnu_network_grp_kfold_35["f1_scores"]).mean(axis=0)

array([0.73186561, 0.57951354])

In [40]:
np.stack(best_swnu_network_grp_kfold_35["precision_scores"]).mean(axis=0)

array([0.78327315, 0.52619194])

In [41]:
np.stack(best_swnu_network_grp_kfold_35["recall_scores"]).mean(axis=0)

array([0.6872133 , 0.64571124])

# w=80

In [11]:
size = 80

## GRP

In [ ]:
(
    swnu_network_grp_kfold_80,
    best_swnu_network_grp_kfold_80,
    _,
    __,
) = swnu_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swnu_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swnu_network_grp_kfold_80.groupby(
    ["dimensions", "swnu_hidden_dim", "ffn_hidden_dim", "dropout_rate", "learning_rate"]
).mean()

In [ ]:
best_swnu_network_grp_kfold_80

In [ ]:
best_swnu_network_grp_kfold_80["f1"].mean()

In [ ]:
best_swnu_network_grp_kfold_80["precision"].mean()

In [ ]:
best_swnu_network_grp_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swnu_network_grp_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swnu_network_grp_kfold_80["recall_scores"]).mean(axis=0)